#### INFO


***Setup***

1. *Runtime ->  Change Runtime Type -> Python 3 + GPU*
2. *Fill in `GDRIVE_DIR` in [code #5] (name of the folder in gdrive that contains your code and data)*

***RUNNING THE SCRIPTS***
1. Mount the drive before the first run. Also, sometimes so you may need to re-mount the drive ([code #6])
2. Tensorboard - should be enough to run the script once; however restart should work OK ([code #7])
3. Training - when re-running the training, you **do not** have to run any of the preceding scripts

***About***

- *[code #1] Install dependencies for google authentification*
- *[code #2] Allow collaboratory to read from & write into your google drive"*
- *[code #3] Download tensorboard accesories*
- *[code #4] Setup paths for tensorboard and your script.*
- *[code #5] Declares the function that will be used to run your tensorflow script.*
- *[code #6] Mounts the google drive to current FS as ./drive.*
- *[code #7] Fires up tensorboard.*
- *[code #8] VM reset script - Kills all processes. Sometimes helps, once the GPU tensorflow stops working, drive fails etc. **! RESETS AUTH !** *
- *[code #9] Sample call to run function that actually runs your script.*


***FINAL NOTES***

Most of this script was copy-pasted from https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

The Tensorflow part is taken from the tutorial from https://github.com/mixuala/colab_utils.git - Special thanks to Štěpán Hojdar.

Feel free to improve this script - just send me suggestions / bug reports / improved scripts and I will upload it here.

Good luck, have fun !

Done mostly by Vladislav Vancák. Some small changes by Petr Houška.

#### SETUP

In [16]:
# DRIVE & AUTH DEPENDENCIES

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpcci0rs4n/pubring.gpg' created
gpg: /tmp/tmpcci0rs4n/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [2]:
# GOOGLE AUTH
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
# TENSORBOARD DEPENDENCIES
!git clone https://github.com/mixuala/colab_utils.git

Cloning into 'colab_utils'...
remote: Counting objects: 216, done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 216 (delta 15), reused 40 (delta 12), pack-reused 171
Receiving objects: 100% (216/216), 60.19 KiB | 3.76 MiB/s, done.
Resolving deltas: 100% (83/83), done.


#### CONFIG

In [0]:
# SCRIPT PROPERTIES
import os

GDRIVE_DIR="DL"

ROOT = %pwd
SCRIPT_PATH = os.path.join(ROOT, 'drive', GDRIVE_DIR)
LOG_DIR = os.path.join(SCRIPT_PATH, 'logs')

In [0]:
# Run script function
import tensorflow as tf
def run_script(name, parameters):
  SCRIPT_FILE= name
  SCRIPT_ARGS= parameters

  if os.path.exists(SCRIPT_PATH):
    os.chdir(SCRIPT_PATH)

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('[WARN]: GPU device not found. Running in CPU mode')
  else:
    print('[INFO]: Found GPU at: {}'.format(device_name))  
  print('<<= <=> =>>')
  
  %run $SCRIPT_FILE $SCRIPT_ARGS

#### MOUNT & TENSORBOARD

In [11]:
# MOUNT DRIVE / CLEAR CACHE
os.chdir(ROOT)
  
if os.path.exists('drive'):
  !google-drive-ocamlfuse -cc

else:
  !mkdir -p drive; 
  !google-drive-ocamlfuse drive

umount: /content/drive: target is busy.
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [18]:
# TENSORBOARD
import os
import sys
import colab_utils.tboard

if os.path.exists(ROOT):
  os.chdir(ROOT)

colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR)

ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://8ef6f39d.ngrok.io


'https://8ef6f39d.ngrok.io'

#### UTILS

In [0]:
# RESETS VM
!kill -9 -1 

#### RUN

In [13]:
run_script(
    "mnist_competition.py",
    "--batch_size 128 --learning_rate 0.001 --learning_rate_final 0.0001 --cnn CB-64-3-1-same,CB-64-3-1-same,M-3-2,C-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,M-3-2,F,R-1024,D --logname 1"
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
Extracting mnist-gan/train-images-idx3-ubyte.gz
Extracting mnist-gan/train-labels-idx1-ubyte.gz
Extracting mnist-gan/t10k-images-idx3-ubyte.gz
Extracting mnist-gan/t10k-labels-idx1-ubyte.gz
0:91.8600
1:99.1000
2:99.5800
3:99.0800
4:99.3400
5:99.8200
6:99.4000
7:99.8800
8:99.6000
9:99.7800
10:99.8600
11:99.8600
12:99.8400
13:99.9200
14:99.7600
15:99.9000
16:99.8400
17:99.8400
18:99.8600
19:99.9000


In [14]:
run_script(
    "mnist_competition.py",
    "--epochs 40 --batch_size 256 --learning_rate 0.001 --learning_rate_final 0.0002 --cnn CB-64-3-1-same,CB-64-3-1-same,M-3-2,C-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,F,R-1024,D --logname 2"
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
Extracting mnist-gan/train-images-idx3-ubyte.gz
Extracting mnist-gan/train-labels-idx1-ubyte.gz
Extracting mnist-gan/t10k-images-idx3-ubyte.gz
Extracting mnist-gan/t10k-labels-idx1-ubyte.gz
0:10.4400
1:95.1200
2:95.9600
3:99.4600
4:99.3800
5:99.5800
6:99.4800
7:94.4000
8:99.4800
9:99.6800
10:99.5600
11:99.0800
12:99.7000
13:99.5600
14:99.2400
15:99.7600
16:97.6000
17:99.8000
18:99.7200
19:99.6200
20:99.6400
21:99.9000
22:99.8800
23:99.9000
24:99.9400
25:99.9600
26:99.9400
27:99.9600
28:99.9600
29:99.9600
30:99.9600
31:99.9400
32:99.9600
33:99.9400
34:99.9200
35:99.9600
36:99.9600
37:99.9400
38:99.9600
39:99.7600


In [17]:
run_script(
    "mnist_competition.py",
    "--epochs 30 --batch_size 256 --learning_rate 0.001 --learning_rate_final 0.0002 --cnn C-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,F,R-1024,D --logname 3"
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
Extracting mnist-gan/train-images-idx3-ubyte.gz
Extracting mnist-gan/train-labels-idx1-ubyte.gz
Extracting mnist-gan/t10k-images-idx3-ubyte.gz
Extracting mnist-gan/t10k-labels-idx1-ubyte.gz
0:9.5400
1:38.2000
2:94.4800
3:98.9000
4:99.4400
5:99.5200
6:89.6800
7:97.9400
8:99.6400
9:99.5600
10:99.3800
11:99.4800
12:98.5600
13:99.6000
14:99.7400
15:99.6200
16:99.7800
17:99.7200
18:99.1200
19:99.3600
20:99.5800
21:99.7600
22:99.8400
23:99.6800
24:99.7800
25:99.5000
26:99.5200
27:99.8600
28:99.8400
29:89.8000


In [22]:
run_script(
    "mnist_competition.py",
    "--epochs 30 --batch_size 256 --learning_rate 0.001 --learning_rate_final 0.0002 --cnn CB-64-3-1-same,CB-64-3-1-same,M-3-2,C-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,F,R-1024,D --logname 2_final --printout True"
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
Extracting mnist-gan/train-images-idx3-ubyte.gz
Extracting mnist-gan/train-labels-idx1-ubyte.gz
Extracting mnist-gan/t10k-images-idx3-ubyte.gz
Extracting mnist-gan/t10k-labels-idx1-ubyte.gz
0:9.5800
1:95.7400
2:98.5800
3:99.4400
4:99.7400
5:99.2200
6:99.3200
7:98.7600
8:99.6400
9:99.7200
10:99.2600
11:99.8000
12:99.7000
13:99.5400
14:99.8200
15:99.7800
16:99.6400
17:99.8200
18:99.9200
19:99.9000
20:99.3200
21:99.1800
22:99.6600
23:99.9200
24:99.8800
25:99.6800
26:99.8600
27:99.5800
28:99.9000
29:99.9200
